# Scraping Financial Statements with Python

#### Import Library

In [2]:
import requests
import lxml
from lxml import html
from datetime import datetime
import pandas as pd
import numpy as np


In [3]:
symbol = 'TLKM.JK'

stocks Telkom

In [4]:
url = 'https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol

In [5]:
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'close',
    'DNT': '1', # Do Not Track Request Header 
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}

In [7]:
page = requests.get(url, headers=headers)
# print(page)

<Response [200]>


In [6]:
tree = html.fromstring(page.content)
# print(tree)
tree.xpath("//h1/text()")

['Perusahaan Perseroan (Persero) PT Telekomunikasi Indonesia Tbk (TLKM.JK)']

In [8]:
table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")

In [11]:
assert len(table_rows) > 0

parsed_rows = []

for table_row in table_rows:
    parsed_row = []
    el = table_row.xpath("./div")
    
    none_count = 0
    
    for rs in el:
        try:
            (text,) = rs.xpath('.//span/text()[1]')
            parsed_row.append(text)
        except ValueError:
            parsed_row.append(np.NaN)
            none_count += 1

    if (none_count < 4):
        parsed_rows.append(parsed_row)

df = pd.DataFrame(parsed_rows)
df

,0,1,2,3,4
0,Breakdown,12/31/2021,12/31/2020,12/31/2019,12/31/2018
1,Total Assets,"276,158,000,000","245,873,000,000","235,057,000,000","205,900,000,000"
2,Total Liabilities Net Minority Interest,"130,986,000,000","125,359,000,000","117,833,000,000","88,894,000,000"
3,Total Equity Gross Minority Interest,"145,172,000,000","120,514,000,000","117,224,000,000","117,006,000,000"
4,Total Capitalization,"157,950,000,000","132,935,000,000","132,085,000,000","130,144,000,000"
5,Common Stock Equity,"121,631,000,000","102,374,000,000","99,796,000,000","98,739,000,000"
6,Capital Lease Obligations,"15,888,000,000","14,877,000,000","17,217,000,000","3,145,000,000"
7,Net Tangible Assets,"108,787,000,000","90,121,000,000","88,355,000,000","92,416,000,000"
8,Working Capital,"-7,407,000,000","-21,971,000,000","-20,432,000,000","-3,479,000,000"
9,Invested Capital,"174,322,000,000","152,219,000,000","149,536,000,000","139,676,000,000"


In [12]:
df = pd.DataFrame(parsed_rows)
df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
df = df.transpose() # Transpose the DataFrame, so that our header contains the account names

# Rename the "Breakdown" column to "Date"
cols = list(df.columns)
cols[0] = 'Date'
df = df.set_axis(cols, axis='columns', inplace=False)

df

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number
1,12/31/2021,"276,158,000,000","130,986,000,000","145,172,000,000","157,950,000,000","121,631,000,000","15,888,000,000","108,787,000,000","-7,407,000,000","174,322,000,000","108,787,000,000","68,579,000,000","14,380,000,000","99,062,217","99,062,217"
2,12/31/2020,"245,873,000,000","125,359,000,000","120,514,000,000","132,935,000,000","102,374,000,000","14,877,000,000","90,121,000,000","-21,971,000,000","152,219,000,000","90,121,000,000","64,722,000,000","29,256,000,000","99,062,217","99,062,217"
3,12/31/2019,"235,057,000,000","117,833,000,000","117,224,000,000","132,085,000,000","99,796,000,000","17,217,000,000","88,355,000,000","-20,432,000,000","149,536,000,000","88,355,000,000","66,957,000,000","31,499,000,000","99,062,217","99,062,217"
4,12/31/2018,"205,900,000,000","88,894,000,000","117,006,000,000","130,144,000,000","98,739,000,000","3,145,000,000","92,416,000,000","-3,479,000,000","139,676,000,000","92,416,000,000","44,082,000,000","23,502,000,000","99,062,217","99,062,217"


In [13]:
df.dtypes

Date                                       object
Total Assets                               object
Total Liabilities Net Minority Interest    object
Total Equity Gross Minority Interest       object
Total Capitalization                       object
Common Stock Equity                        object
Capital Lease Obligations                  object
Net Tangible Assets                        object
Working Capital                            object
Invested Capital                           object
Tangible Book Value                        object
Total Debt                                 object
Net Debt                                   object
Share Issued                               object
Ordinary Shares Number                     object
dtype: object

In [14]:
df

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number
1,12/31/2021,"276,158,000,000","130,986,000,000","145,172,000,000","157,950,000,000","121,631,000,000","15,888,000,000","108,787,000,000","-7,407,000,000","174,322,000,000","108,787,000,000","68,579,000,000","14,380,000,000","99,062,217","99,062,217"
2,12/31/2020,"245,873,000,000","125,359,000,000","120,514,000,000","132,935,000,000","102,374,000,000","14,877,000,000","90,121,000,000","-21,971,000,000","152,219,000,000","90,121,000,000","64,722,000,000","29,256,000,000","99,062,217","99,062,217"
3,12/31/2019,"235,057,000,000","117,833,000,000","117,224,000,000","132,085,000,000","99,796,000,000","17,217,000,000","88,355,000,000","-20,432,000,000","149,536,000,000","88,355,000,000","66,957,000,000","31,499,000,000","99,062,217","99,062,217"
4,12/31/2018,"205,900,000,000","88,894,000,000","117,006,000,000","130,144,000,000","98,739,000,000","3,145,000,000","92,416,000,000","-3,479,000,000","139,676,000,000","92,416,000,000","44,082,000,000","23,502,000,000","99,062,217","99,062,217"


## Scrape some balance sheet data

In [9]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

def get_page(url):
    # Set up the request headers that we're going to use, to simulate
    # a request by the Chrome browser. Simulating a request from a browser
    # is generally good practice when building a scraper
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Connection': 'close',
        'DNT': '1', # Do Not Track Request Header 
        'Pragma': 'no-cache',
        'Referrer': 'https://google.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
    }

    return requests.get(url, headers=headers)

def parse_rows(table_rows):
    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)
            
    return pd.DataFrame(parsed_rows)

def clean_data(df):
    df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
    df = df.transpose() # Transpose the DataFrame, so that our header contains the account names
    
    # Rename the "Breakdown" column to "Date"
    cols = list(df.columns)
    cols[0] = 'Date'
    df = df.set_axis(cols, axis='columns', inplace=False)
    
    numeric_columns = list(df.columns)[1::] # Take all columns, except the first (which is the 'Date' column)

    for column_index in range(1, len(df.columns)): # Take all columns, except the first (which is the 'Date' column)
        df.iloc[:,column_index] = df.iloc[:,column_index].str.replace(',', '') # Remove the thousands separator
        df.iloc[:,column_index] = df.iloc[:,column_index].astype(np.float64) # Convert the column to float64
        
    return df

def scrape_table(url):
    # Fetch the page that we're going to parse
    page = get_page(url);

    # Parse the page with LXML, so that we can start doing some XPATH queries
    # to extract the data that we want
    tree = html.fromstring(page.content)

    # Fetch all div elements which have class 'D(tbr)'
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")
    
    # Ensure that some table rows are found; if none are found, then it's possible
    # that Yahoo Finance has changed their page layout, or have detected
    # that you're scraping the page.
    assert len(table_rows) > 0
    
    df = parse_rows(table_rows)
    df = clean_data(df)
        
    return df


In [10]:
symbol = 'TLKM.JK'
df_balance_sheet = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol)

In [11]:
scrape_table('https://finance.yahoo.com/quote/' + symbol + '/financials?p=' + symbol)

,Date,Total Revenue,Cost of Revenue,Gross Profit,Operating Expense,Operating Income,Net Non Operating Interest Income Expense,Other Income Expense,Pretax Income,Tax Provision,...,Net Interest Income,EBIT,Reconciled Cost of Revenue,Reconciled Depreciation,Net Income from Continuing Operation Net Minority Interest,Total Unusual Items Excluding Goodwill,Total Unusual Items,Normalized EBITDA,Tax Rate for Calcs,Tax Effect of Unusual Items
1,ttm,1.457130e+11,4.398100e+10,1.017320e+11,5.832300e+10,4.340900e+10,-3.620000e+09,3.585000e+09,4.337400e+10,8.640000e+09,...,-3.620000e+09,4.765600e+10,4.398100e+10,3.400200e+10,2.573600e+10,3.471000e+09,3.471000e+09,7.818700e+10,0.0,691415133.0
2,12/31/2021,1.432100e+11,4.331400e+10,9.989600e+10,5.588700e+10,4.400900e+10,-3.836000e+09,3.566000e+09,4.373900e+10,9.640000e+09,...,-3.836000e+09,4.813300e+10,4.331400e+10,3.171400e+10,2.487700e+10,3.482000e+09,3.482000e+09,7.636500e+10,0.0,767426782.0
3,12/31/2020,1.364470e+11,3.998100e+10,9.646600e+10,5.336100e+10,4.310500e+10,-3.802000e+09,-1.560000e+08,3.914700e+10,9.257000e+09,...,-3.802000e+09,4.374900e+10,3.998100e+10,2.892500e+10,2.105200e+10,-8.490000e+08,-8.490000e+08,7.352300e+10,0.0,-200761054.0
4,12/31/2019,1.355570e+11,4.253000e+10,9.302700e+10,4.983900e+10,4.318800e+10,-4.357000e+09,-5.320000e+08,3.829900e+10,1.043900e+10,...,-4.357000e+09,4.375100e+10,4.253000e+10,2.720400e+10,1.906800e+10,-1.261000e+09,-1.261000e+09,7.221600e+10,0.0,-343705554.0
5,12/31/2018,1.307880e+11,4.817600e+10,8.261200e+10,4.521500e+10,3.739700e+10,-2.509000e+09,1.189000e+09,3.607700e+10,9.366000e+09,...,-2.509000e+09,3.960000e+10,4.817600e+10,2.144200e+10,1.780200e+10,7.100000e+07,7.100000e+07,6.097100e+10,0.0,18432408.0


In [19]:
scrape_table('https://finance.yahoo.com/quote/' + symbol + '/cash-flow?p=' + symbol)

,Date,Cash Flows from Used in Operating Activities Direct,Investing Cash Flow,Financing Cash Flow,End Cash Position,Capital Expenditure,Issuance of Capital Stock,Issuance of Debt,Repayment of Debt,Free Cash Flow
1,ttm,7.021200e+10,-3.642700e+10,-3.036900e+10,4.002900e+10,-3.564900e+10,1.850800e+10,4.383500e+10,-5.157900e+10,3.456300e+10
2,12/31/2021,6.835300e+10,-3.791400e+10,-1.277500e+10,3.831100e+10,-3.276800e+10,1.846300e+10,4.661200e+10,-4.796500e+10,3.558500e+10
3,12/31/2020,6.531700e+10,-3.509900e+10,-2.791000e+10,2.058900e+10,-3.194100e+10,NaN,2.446900e+10,-2.933900e+10,3.337600e+10
4,12/31/2019,5.896600e+10,-3.587500e+10,-2.217500e+10,1.824100e+10,-3.731000e+10,NaN,2.652400e+10,-2.291100e+10,2.165600e+10
5,12/31/2018,4.567100e+10,-3.509000e+10,-1.845800e+10,1.743500e+10,-3.483400e+10,3.400000e+07,3.536400e+10,-2.711300e+10,1.083700e+10


In [12]:
def scrape(symbol):
    print('Attempting to scrape data for ' + symbol)

    df_balance_sheet = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol)
    df_balance_sheet = df_balance_sheet.set_index('Date')

    df_income_statement = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/financials?p=' + symbol)
    df_income_statement = df_income_statement.set_index('Date')
    
    df_cash_flow = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/cash-flow?p=' + symbol)
    df_cash_flow = df_cash_flow.set_index('Date')
    
    df_joined = df_balance_sheet \
        .join(df_income_statement, on='Date', how='outer', rsuffix=' - Income Statement') \
        .join(df_cash_flow, on='Date', how='outer', rsuffix=' - Cash Flow') \
        .dropna(axis=1, how='all') \
        .reset_index()
            
    df_joined.insert(1, 'Symbol', symbol)
    
    return df_joined

In [13]:
def scrape_multi(symbols):
    return pd.concat([scrape(symbol) for symbol in symbols], sort=False)

In [14]:
symbols = ['TLKM.JK', 'BBCA.JK']
df_combined = scrape_multi(symbols)

Attempting to scrape data for TLKM.JK
Attempting to scrape data for BBCA.JK


In [16]:
date = datetime.today().strftime('%Y-%m-%d')
writer = pd.ExcelWriter('TLKM&BBCA-Finance-Scrape-' + date + '.xlsx')
df_combined.to_excel(writer)
writer.save()